# Шаг 1: Загрузка всех данных и объединение

In [6]:
import pandas as pd

# Общий список колонок

columns = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
    'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
    'ca', 'thal', 'target'
]

# Ссылки на датасеты
urls = {
    'cleveland': 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data',
    'hungarian': 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data',
    'switzerland': 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data',
    'va': 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.va.data'
}

# загрузка и объединение данных
dfs = []
for clinic, url in urls.items():
    temp_df = pd.read_csv(url, names=columns)
    temp_df.replace('?', pd.NA, inplace=True)
    print(f'{clinic.capitalize()} загружен: {temp_df.shape[0]} записей.')
    dfs.append(temp_df)

# объединяем в один датафрейм
df_full = pd.concat(dfs, ignore_index=True)

# Заполнение пропусков медианой
# Корректное заполнение пропусков медианой без chained assignment
for col in df_full.columns:
    df_full[col] = pd.to_numeric(df_full[col], errors='coerce')
    median = df_full[col].median()
    df_full[col] = df_full[col].fillna(median)

# Проверим ещё раз отсутствие пропусков
print('Пропущенных значений после заполнения:', df_full.isnull().sum().sum())


# Бинаризация целевой переменной
df_full['target'] = df_full['target'].apply(lambda x: 1 if int(x) > 0 else 0)

# Проверка итогового датасета
print('Итоговый размер датасета:', df_full.shape)
print(df_full.info())
print(df_full.head())

Cleveland загружен: 303 записей.
Hungarian загружен: 294 записей.
Switzerland загружен: 123 записей.
Va загружен: 200 записей.
Пропущенных значений после заполнения: 0
Итоговый размер датасета: (920, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       920 non-null    float64
 1   sex       920 non-null    float64
 2   cp        920 non-null    float64
 3   trestbps  920 non-null    float64
 4   chol      920 non-null    float64
 5   fbs       920 non-null    float64
 6   restecg   920 non-null    float64
 7   thalach   920 non-null    float64
 8   exang     920 non-null    float64
 9   oldpeak   920 non-null    float64
 10  slope     920 non-null    float64
 11  ca        920 non-null    float64
 12  thal      920 non-null    float64
 13  target    920 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 100.8 KB
None
    age  

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# One-Hot Encoding категориальных признаков
df_encoded = pd.get_dummies(df_full, columns=['cp', 'thal', 'restecg', 'slope'])

# Делим на признаки (X) и целевую переменную (y)
X = df_encoded.drop('target', axis=1)
y = df_encoded['target']

# Разделение данных на обучающую и тестовую выборки (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Масштабирование числовых признаков
scaler = StandardScaler()

# Числовые признаки для масштабирования
numeric_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']

# Применяем масштабирование
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

# Проверка размеров
print('Train shape:', X_train.shape, y_train.shape)
print('Test shape:', X_test.shape, y_test.shape)


Train shape: (736, 22) (736,)
Test shape: (184, 22) (184,)


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# параметры для поиска лучших гиперпараметров
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# GridSearchCV
grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), 
                    param_grid, cv=5, scoring='accuracy')

# обучение
grid.fit(X_train, y_train)

# Лучшие параметры и точность модели
print("Лучшие параметры:", grid.best_params_)
print("Лучшая точность (CV):", grid.best_score_)

# Тестирование модели на тестовой выборке
best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_test)

# Оценка результатов
from sklearn.metrics import accuracy_score, classification_report

print("\nТочность на тестовых данных:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))


Лучшие параметры: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Лучшая точность (CV): 0.8178985107556537

Точность на тестовых данных: 0.8260869565217391
              precision    recall  f1-score   support

           0       0.84      0.76      0.79        82
           1       0.82      0.88      0.85       102

    accuracy                           0.83       184
   macro avg       0.83      0.82      0.82       184
weighted avg       0.83      0.83      0.82       184



In [9]:
import joblib

# Сохраняем модель в файл
joblib.dump(best_model, 'heart_disease_model.pkl')

# Сохраняем scaler
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']